<h1><center>  卖方研究中的共识 -  Nucleus API用例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有 </center></h1>
<h1><center>  免责声明和服务条款可通过www.sumup.ai获取 </center></h1>


#  



## 目标: 
-	衡量卖方分析师对特定公司的共识


## 数据:
-	卖方的股票研究报告



## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files leverages api_instance.post_upload_file with parallel execution to speed-up the dataset creation


-	文件情绪 API
 - 	*api_instance.post_doc_sentiment_api(payload)*
 

-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	DocInfo API
 - 	*api_instance.post_doc_info(payload)*


## 方法:

### 1.	数据集准备
-	创建包含所有相关文档的Nucleus数据集

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
print('--------- Append all files from local folder to dataset in parallel -----------')
folder = 'Sellside_research'         
dataset = 'Sellside_research'# 字符串 | 文件的目标数据集.

# 以递归方式从文件夹构建文件。  
# 其中每个项目采用以下格式：
# {'filename': filename,   # 要上传的文件名。必填
#  'metadata': {           # 该文件的元数据。选填
#      'key1': val1,       # 密钥就可以取任意名称，名字符合以下字符
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #如果 Path(file).suffix == '.pdf': # 也支持.txt .doc .docx .rtf .html .csv 格式
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'ticker': 'AAPL',
                                      'company': 'Apple',
                                      'bank': 'Credit Suisse',
                                      'category': 'sell side research'
                                      'date': '2019-01-01'}}
            file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

-	对于给定日期，仅保留在其回顾期间发布的文档的子集

**这可以直接在API中完成，以执行内容分析，如下所示**



### 2.	衡量一只股票在卖方的共识
- 单选一只个股的所有文件


- 衡量每个文件的情绪


- 推导出共识值，以及与该情绪相似的银行的比例
 

In [ ]:
# 提取一个公司的所有文档
import numpy as np

payload = nucleus_api.DocInfo(dataset='Sellside_research',
                             metadata_selection={'ticker': 'AAPL'})
api_response = api_instance.post_doc_info(payload)

doc_list = []
for res in api_response.result:        
    doc_list.append(res.title) 

doc_list = np.unique(doc_list)

print('-------- Get the sentiment of each document ----------------')
reports_sentiment = []
for i in range(len(company_list)):
    payload = nucleus_api.DocumentSentimentModel(dataset='Sellside_research', 
                                                doc_title=doc_list[i], 
                                                custom_stop_words="", 
                                                num_topics=10, 
                                                num_keywords=10)
    try:
        api_response = api_instance.post_doc_sentiment_api(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:       
        reports_sentiment.append(api_response.result.sentiment)

# 推导出公司在卖方报告中的共识
company_consensus = np.asscalar(max(reports_sentiment > 0, reports_sentiment < 0, reports_sentiment == 0) / len(reports_sentiment))


-	如有多个感兴趣的公司，每家公司重复以上步骤

In [ ]:
import numpy as np

# 感兴趣的公司名单
company_list = ['AAPL', 'GOOG', 'FB', 'BABA', 'NFLX']

# 每一个公司对应一个共识
company_consensus = []
for i in range(len(company_list)):  
    # 获取讨论该公司的所有文档
    payload = nucleus_api.DocInfo(dataset='Sellside_research',
                                 metadata_selection={'ticker': company_list[i]})
    api_response = api_instance.post_doc_info(payload)

    doc_list = []
    for res in api_response.result:        
        doc_list.append(res.title) 

    doc_list = np.unique(doc_list)

    # 获取每个文档的情绪
    reports_sentiment = []
    for j in range(len(company_list)):
        payload = nucleus_api.DocumentSentimentModel(dataset='Sellside_research', 
                                                    doc_title=doc_list[j], 
                                                    custom_stop_words="", 
                                                    num_topics=10, 
                                                    num_keywords=10)
        try:
            api_response = api_instance.post_doc_sentiment_api(payload)
            api_ok = True
        except ApiException as e:
            api_error = json.loads(e.body)
            print('ERROR:', api_error['message'])
            api_ok = False

        if api_ok:   
            reports_sentiment.append(api_response.result.sentiment)

    # 推断出该公司在卖方报告中的共识
    company_consensus.append(np.asscalar(max(reports_sentiment > 0, reports_sentiment < 0, reports_sentiment == 0) / len(reports_sentiment)))

### 3.	结果
-	根据该公司的在这个行业中的共识值，绘制起该公司的共识价对比时间序列，或绘制该公司的共识价对比股票收益/波动率

### 4.	微调

#### a.	量身定制模型

-   通过使用Document Sentiment API中的`custom_stop_words`参数，排除无关的主题以定制目标公司的共识


-	有针对性的筛选该公司一部分文档中的关键主题



In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Sellside_research',                       
                            query='',                       
                            num_topics=20, 
                            num_keywords=5,
                            metadata_selection={'ticker': 'AAPL'})
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

然后，您可以通过创建custom_stop_words变量来定制公司的共识。 例如，如下所示，定义一个变量，并将其加入上面第2节 “ 衡量一只股票在卖方的共识” 代码中的payload：

In [ ]:
custom_stop_words = ["call","report"] # str | 停用词列表。 （可选）

#### b.探索文档类型，回顾期，提取的主题数量的影响
**num_topics**: 您可以通过更改元数据（payload）中的变量 `num_topics`来计算公司对不同广度主题的共识。
- 在建立共识时，较大的值将包含更广泛的主题/内容，而较小的值将提供具有更简明的主题。
- 如果num_topics太大，一些非常边缘的话题可能会在衡量公司共识时带来很多噪音。

**Document types**: 您可以调查公司的共识如何随着不同类型的文档而变化，例如卖方研究，新闻或公司出版物。 在不同的数据集上重新运行第2节的主要代码。 您还可以使用所有可用类型的文档构建数据集，然后使用`metadata_selection`参数选择特定类型：

In [ ]:
metadata_selection = {"category": "News"}   # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)

### 5.	下一步
-	可能的扩展：针对不同的行业部门重复上述任务
-   汇总某个行业中每家公司的情绪，以推断行业共识

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 